### 라이브러리 로드

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch

from kobert import get_pytorch_kobert_model
from kobert import get_tokenizer

import gluonnlp as nlp

import numpy as np

### 1. 모델 불러오기

##### 데이터셋 클래스 정의 
- 모델 학습할 때 사용한 데이터셋 클래스 정의, 입력 데이터와 레이블을 적절한 형태로 변환하여 제공하기 위함

In [2]:
# dataset
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

##### 하이퍼파라미터 및 device 지정
- 모델 학습 시 사용한 하이퍼파라미터 및 장치 설정을 동일하게 지정

In [4]:
# hyperparameter
seed = 0
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
device = 'cuda' if torch.cuda.is_available() else 'cpu'

##### 모델 클래스 정의와 인스턴스화
- 학습할 때 사용한 모델 클래스 정의 및 해당 클래스를 사용한 모델 객체 인스턴스화를 통해 모델 구조와 설정 재현

In [5]:
# KoBERT 모델
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        out = self.dropout(pooler)
    else:
        out = pooler
    return self.classifier(out)

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)  # 모델 클래스를 정의하고 인스턴스화

using cached model. /home/gksthwls3566/.cache/kobert_v1.zip
using cached model. /home/gksthwls3566/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/gksthwls3566/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


##### 저장된 모델 가중치 로드
- 학습된 모델 가중치

In [7]:
# 저장된 모델의 가중치 로드
model.load_state_dict(torch.load("naverShoppingReview_state_dict.pt"))

<All keys matched successfully>

### 2. 모델 적용 예시
- 클래스는 총 2개(부정, 긍정)이며 부정, 긍정에 대한 로짓값 비교
- 각 클래스에 대한 로짓값을 비교해 더 높은 값의 클래스를 예측해줌

In [68]:
# 예측 함수 생성
def predict(predict_sentence):

    data = [predict_sentence, 0]
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_loader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_loader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            if np.argmax(logits) == 0:
                test_eval.append("부정")
            else:
                test_eval.append("긍정")
    return test_eval[0]

In [56]:
predict("해양 산업에 대한 정부의 지원이 지속해서 상승세를 띄고 있습니다.")

'긍정'

In [57]:
predict("해양 산업에 대한 정부의 지원이 지속해서 하락세를 띄고 있습니다.")

'부정'

- 항만 기사 하루치 가져와서 긍부정 진행

In [86]:
import pandas as pd

test = pd.DataFrame()
for i in range(1,7):
  a = pd.read_csv("./크롤링/항만/2003-05-{:02d}_항만.csv".format(i))
  print(a)
  test = pd.concat([test,a])

                                   기사제목     언론사    년도  월   
0                [부산/경남]부산항만공사 내년 공식 출범    동아일보  2003  5  \
1                   “인천공항 고속도 통행료 인하해야”    동아일보  2003  5   
2                 광양컨 정보 한눈에 홈페이지 구축 운영    무등일보  2003  5   
3                       인천항사스의심환자발생사스비상    중부일보  2003  5   
4                  <2003 핫 이슈> 활개치는 총기류    경향신문  2003  5   
5                       `민자시설 사용료 인하필요`    매일경제  2003  5   
6       "노동운동 정치적 이용 안타까워" 항운노력 최봉홍 위원장    문화일보  2003  5   
7                 SOC 활성화위해 법인세 면제해야-상의   머니투데이  2003  5   
8                       광양항 인터넷 홈페이지 개설    전남일보  2003  5   
9                     마산항 부두기능 내년 전면 조정  경남도민일보  2003  5   
10                      관행어업' 인정돼야 보상가능  경남도민일보  2003  5   
11  허성관 해양부 장관 PA법 통과 회견 ＇항만공사 후속조치 최선＇    국제신문  2003  5   
12                            PA법 국회 통과    국제신문  2003  5   
13                                항만공사란    부산일보  2003  5   
14                      PA법 국회 통과 의미·내용    부산일보  2003  5   
15                               새로운 출발 

In [87]:
result_title, result_ariticle = [predict(a) for a in test['기사제목']], [predict(b) for b in test['기사내용']]


In [88]:
test['기사제목_감정분석'] = result_title
test['기사내용_감정분석'] = result_ariticle

In [89]:
test.reindex(columns=['기사제목','기사제목_감정분석','기사내용', '기사내용_감정분석','언론사', '년도', '월', 'url'])

,기사제목,기사제목_감정분석,기사내용,기사내용_감정분석,언론사,년도,월,url
0,[부산/경남]부산항만공사 내년 공식 출범,긍정,부산의 숙원인 항만공사가 내년에 공식 출범해 본격적인 부산항 운영을 시작한다. 국...,긍정,동아일보,2003,5,https://www.donga.com/news/article/all/2003050...
1,“인천공항 고속도 통행료 인하해야”,부정,"부가세와 법인세 인하를 통해 도로, 항만 등 민간자본 유치 사회간접자본(SOC) 사...",부정,동아일보,2003,5,https://www.donga.com/news/article/all/2003050...
2,광양컨 정보 한눈에 홈페이지 구축 운영,긍정,광양컨테이너항의 항만정보를 인터넷으로 제공하기 위한 ‘광양항 홈페이지’가 구축돼 운...,긍정,무등일보,2003,5,http://www.honam.co.kr/detail/GQFJv6/92679
3,인천항사스의심환자발생사스비상,부정,인천공항을 통해 입국한 국내 첫 사스(SARS 중증급성호흡기증후군) 추정환자 발생에...,부정,중부일보,2003,5,www.joongboo.com/?mod=news&act=articleView&idx...
4,<2003 핫 이슈> 활개치는 총기류,긍정,■잇단 총기사고로 본 실태최근 총기사고가 잇따르면서 우리나라도 더 이상 총기사고 안...,부정,경향신문,2003,5,NaN
...,...,...,...,...,...,...,...,...
21,성장률 3%후반∼4%초반 전망,부정,최근의 경기위축은 대내외적인 불안요인들로 인해 상반기중 지속되고 경제성장률은 3%후...,부정,파이낸셜뉴스,2003,5,NaN
22,"국내기관, 적극적 재정정책 주문(2보)",긍정,국내연구기관들은 내수위축을 보완하기 위해 우선 적극적인 재정정책 검토를 주문.. 특...,부정,머니투데이,2003,5,NaN
23,[여의나루] 사스 검역에 휴대용 열추적기 동원,부정,국무총리실에 설치돼 있는 사스(SARS·급성중증호흡기증후군) 대책본부가 요즘 사스환...,부정,국민일보,2003,5,NaN
24,[거시경제 점검회의 뭘 논의했나]“경제 불안심리 안정에 주력”,부정,정부는 올 성장률이 4% 안팎에 머물 것으로 보임에 따라 적극적인 재정정책과 금리인...,긍정,파이낸셜뉴스,2003,5,NaN


In [91]:
test.to_csv("./kobert_030501_030507.csv")